# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [2]:
import requests

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [3]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)

response.json()

ConnectionError: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("HTTPSConnection(host='api.openai.com', port=443): Failed to resolve 'api.openai.com' ([Errno 11001] getaddrinfo failed)"))

In [4]:
response.json()["choices"][0]["message"]["content"]

NameError: name 'response' is not defined

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [5]:
# Create OpenAI client

from openai import OpenAI
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content



'Fun fact: Honey never spoils. Archaeologists have found edible honey in ancient Egyptian tombs that are over 3,000 years old. Its low water content and acidic pH help preserve it.'

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

## THIS IS OPTIONAL - but if you wish to try out Google Gemini, please visit:

https://aistudio.google.com/

And set up your API key at

https://aistudio.google.com/api-keys

And then add your key to the `.env` file, being sure to Save the .env file after you change it:

`GOOGLE_API_KEY=AIz...`


In [6]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

load_dotenv(override=True)

google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini


In [7]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(model="gemini-2.5-flash-lite", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}]

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [4]:
import requests
requests.get("http://localhost:11434").content

b'Ollama is running'

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [5]:
!ollama pull llama3.2

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest 
pulling dde5aa3fc5ff: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 2.0 GB                         
pulling 966de95ca8a6: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.4 KB                         
pulling fcc5a6bec9da: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 7.7 KB                         
pulling a70ff7e570d9: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 6.0 KB                         
pulling 56bb8bd477a

In [9]:
!pip install -U openai


In [10]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
from openai import OpenAI
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [11]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'Here\'s one:\n\nDid you know that there is a type of jellyfish that is immortal? The Turritopsis dohrnii, also known as the "immortal jellyfish," can transform its body into a young state through a process called transdifferentiation. This means it can essentially revert back to its polyp stage and grow back into an adult again, making it theoretically immortal!'

In [15]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest 
pulling aabd4debf0c8: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 GB                         
pulling c5ad996bda6e: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  556 B                         
pulling 6e4c38e1172f: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 KB                         
pulling f4d24e9138dd: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  148 B                         
pulling a85fe2a2e58e: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [18]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

b = response.choices[0].message.content
print(b)

Sure! Here's a fun fact: 

The fastest human projectile ever shot, though it wasn't a projectile in the traditional sense, is around 3.5 kilometers per second (about 7.8 miles per hour). This speed was achieved by a bullet from a marksman called Mark Forest Ray 9 from North Carolina in 1964.


# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [27]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

In [28]:
# Let's try out this utility

ed = fetch_website_contents("https://marufmullah50.vercel.app/")
print(ed)

Md. Maruf Mullah — Mechanical Engineer

About Me
Education
Research
Projects
Technical Skills
Experience & Achievements
Contact
Download CV
Md. Maruf Mullah
Mechanical Engineer & ML/DL Enthusiast
Bridging classical engineering with computational tools, machine learning, and experimental research.
Learn More
Get In Touch
About Me
I'm
Md. Maruf Mullah
, a recent Mechanical Engineering graduate from the Military Institute of Science and Technology (MIST). My journey combines traditional engineering fundamentals with cutting-edge computational approaches.
And my work focuses on materials engineering, additive manufacturing, and data-driven modeling. I apply machine learning and deep learning techniques to predict material behavior, analyze performance patterns, and support decision-making in engineering processes.

I have experience in CAD/CAM design, FDM 3D printing workflows, and prototyping, along with developing small autonomous and robotic systems. My projects include surface roughnes

In [33]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [34]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

In [35]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [36]:

from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"  # dummy
)

def summarize(url):
    website = fetch_website_contents(url)

    response = client.chat.completions.create(
        model="deepseek-r1:1.5b",
        messages=messages_for(website)
    )

    return response.choices[0].message.content

In [37]:
summarize("https://marufmullah50.vercel.app")

'A website focused on the intricate web of programming content analysis, highlighting areas like AI, materials engineering and machine learning, perhaps even connecting these skills to interesting trends or niche topics in computing. The tone is snarky, using humor to vent frustration at not finding as much beyond their own analytical interests, blending that with a critique of common tech trends.'

In [38]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [40]:
display_summary("https://marufmullah50.vercel.app/")

Md Maruf Mbull is a Mechanical Engineer with a passion for deep learning and machine learning techniques, applying them to solve practical engineering problems. His work on predicting surface roughness and classifying fish images from photos shows his creativity and dedication. He has an upcoming project on wind forecasting using advanced neural networks! Keep up the impressive research in mechanical engineering and data science!

In [42]:
import requests
from bs4 import BeautifulSoup

class Website:
    """
    Website scraper
    """
    url: str
    title: str
    text: str

    def __init__(self, url: str):
        self.url = url

        headers = {
            "User-Agent": "Mozilla/5.0"
        }

        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, "html.parser")

        self.title = soup.title.string.strip() if soup.title else "No title found"

        # Remove irrelevant tags
        for irrelevant in soup(["script", "style", "img", "input", "noscript"]):
            irrelevant.decompose()

        body = soup.body
        self.text = body.get_text(separator="\n", strip=True) if body else ""


In [43]:
maruf = Website("https://marufmullah50.vercel.app/")
print(f"Title is: {maruf.title}")
print("\n")
print("Texts are:")
print(maruf.text)

Title is: Md. Maruf Mullah — Mechanical Engineer


Texts are:
About Me
Education
Research
Projects
Technical Skills
Experience & Achievements
Contact
Download CV
Md. Maruf Mullah
Mechanical Engineer & ML/DL Enthusiast
Bridging classical engineering with computational tools, machine learning, and experimental research.
Learn More
Get In Touch
About Me
I'm
Md. Maruf Mullah
, a recent Mechanical Engineering graduate from the Military Institute of Science and Technology (MIST). My journey combines traditional engineering fundamentals with cutting-edge computational approaches.
And my work focuses on materials engineering, additive manufacturing, and data-driven modeling. I apply machine learning and deep learning techniques to predict material behavior, analyze performance patterns, and support decision-making in engineering processes.

I have experience in CAD/CAM design, FDM 3D printing workflows, and prototyping, along with developing small autonomous and robotic systems. My projects in

In [44]:
# Step 1: Create your prompts
system_prompt = "You are a highly c technical assistant in Web scapping.For web-scraped or unstructured text, extract only meaningful human-readable content while ignoring navigation, ads, footers, scripts, and other irrelevant elements, and summarize or restructure it clearly when requested."

def user_prompt(website):
    user_prompt = f"You are looking at a website titled (website.title)"
    user_prompt += "The contents of this website is as follows;\
please provide a short summary of this website in Markdown.\
If it includes news or projects , then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt



In [45]:
user_prompt(maruf)

"You are looking at a website titled (website.title)The contents of this website is as follows;please provide a short summary of this website in Markdown.If it includes news or projects , then summarize these too.\n\nAbout Me\nEducation\nResearch\nProjects\nTechnical Skills\nExperience & Achievements\nContact\nDownload CV\nMd. Maruf Mullah\nMechanical Engineer & ML/DL Enthusiast\nBridging classical engineering with computational tools, machine learning, and experimental research.\nLearn More\nGet In Touch\nAbout Me\nI'm\nMd. Maruf Mullah\n, a recent Mechanical Engineering graduate from the Military Institute of Science and Technology (MIST). My journey combines traditional engineering fundamentals with cutting-edge computational approaches.\nAnd my work focuses on materials engineering, additive manufacturing, and data-driven modeling. I apply machine learning and deep learning techniques to predict material behavior, analyze performance patterns, and support decision-making in enginee

In [46]:
print(user_prompt(maruf))

You are looking at a website titled (website.title)The contents of this website is as follows;please provide a short summary of this website in Markdown.If it includes news or projects , then summarize these too.

About Me
Education
Research
Projects
Technical Skills
Experience & Achievements
Contact
Download CV
Md. Maruf Mullah
Mechanical Engineer & ML/DL Enthusiast
Bridging classical engineering with computational tools, machine learning, and experimental research.
Learn More
Get In Touch
About Me
I'm
Md. Maruf Mullah
, a recent Mechanical Engineering graduate from the Military Institute of Science and Technology (MIST). My journey combines traditional engineering fundamentals with cutting-edge computational approaches.
And my work focuses on materials engineering, additive manufacturing, and data-driven modeling. I apply machine learning and deep learning techniques to predict material behavior, analyze performance patterns, and support decision-making in engineering processes.

I h

In [55]:
print(system_prompt)

You are a highly c technical assistant in Web scapping.For web-scraped or unstructured text, extract only meaningful human-readable content while ignoring navigation, ads, footers, scripts, and other irrelevant elements, and summarize or restructure it clearly when requested.


In [70]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display


In [71]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL ="llama3.2"

In [72]:
messages=[
    {"role":"user","content": "Describe About Mechanical Engineering and Physics Informed Neural Networks in Mechanical Engineering" }  
]

In [73]:
payload = {
    "model": MODEL,
    "messages": messages,
    "stream": False
}

In [75]:
response = requests.post(OLLAMA_API, json =payload, headers= HEADERS)
print(response.json()['message']['content'])

Mechanical Engineering and Physics-Informed Neural Networks (PINNs) are two closely related fields that are revolutionizing the way we approach problems in mechanical engineering.

**Mechanical Engineering**

Mechanical engineering is a branch of engineering that deals with the design, construction, and use of machines, mechanisms, and systems that convert energy into useful forms. Mechanical engineers apply principles from physics and mathematics to develop solutions for complex problems in fields such as:

1. Thermal Systems (heat transfer, thermodynamics)
2. Fluid Dynamics (pumps, turbines, compressors)
3. Mechanics of Materials (stress, strain, fracture mechanics)
4. Vibration Analysis (vibration control, structural dynamics)
5. Control Systems (mechanical control systems, robotics)

Mechanical engineers use a range of tools and techniques to analyze and design mechanical systems, including finite element analysis, computational fluid dynamics, and simulation software.

**Physics-I

In [76]:
message1=[
    {"role":"user","content": "Describe About Blackhole" }  
]

In [77]:
#Using python Library
import ollama
response = ollama.chat(model = MODEL, messages=message1)
print(response['message']['content'])

A black hole is a region in space where the gravitational pull is so strong that nothing, including light, can escape. It is formed when a massive star collapses in on itself and its gravity becomes so strong that it warps the fabric of spacetime around it.

Here's a more detailed explanation:

**How are Black Holes Formed?**

Black holes are typically formed when a massive star runs out of fuel and dies. If the star is massive enough (about 3-4 times the size of the sun), its gravity will collapse in on itself, causing a supernova explosion. If the star is even more massive (about 10-20 times the size of the sun), its gravity will be so strong that it will warp spacetime around it, creating a singularity at its center.

**What are the Characteristics of Black Holes?**

Black holes have several characteristics that make them fascinating objects of study:

1. **Event Horizon**: The point of no return around a black hole is called the event horizon. Once something crosses the event horiz